# Imports

In [ ]:
from modules.data import  data_reader, data_loader
from modules.sr import result_saver
from modules.models import lag_llama
from modules.experiment.tscv import get_tscv_results, get_summary, extract_metrics
from modules.visualization import graphs
from modules.fine_tuning import lag_llama_ft
from modules.models import prpht


# Notes

Earliest monthly data for S&P500 is 1985
FTSE 100 also 1985
NASDAQ also 1985


In [8]:
from modules.data import data_loader

a = data_loader.get_data(type = "index", ticker="S&P 500", frequency = "weekly", start = "2017-01-01", end = "2022-01-01", rtrn = True)

[*********************100%%**********************]  1 of 1 completed


In [10]:
a[:200]

,ds,y
1,2017-01-08,-0.001028
2,2017-01-15,-0.001464
3,2017-01-22,0.010294
4,2017-01-29,0.001190
5,2017-02-05,0.008131
...,...,...
196,2020-10-04,0.038442
197,2020-10-11,0.001918
198,2020-10-18,-0.005287
199,2020-10-25,-0.056395


In [ ]:
cc = data_loader.get_data(type = "cc", frequency = "weekly", ticker = "Social")

In [ ]:
cc

In [ ]:
a = data_loader.get_data(type = "fx", ticker = "USD/GBP", frequency = "daily", rtrn = False)

In [ ]:
a

In [ ]:
cc["Date", "Aggregate"]

In [ ]:
a.head(10)

In [ ]:
from modules.data import data_loader

a = data_loader.get_data(type = "index",ticker="NASDAQ", frequency = "daily", start = "2022-01-01", end = "2024-01-01", rtrn = True)

In [ ]:
a

In [ ]:
b = data_loader.get_data(type = "index",ticker="S&P 500", frequency = "daily", start = "2022-01-01", end = "2024-01-01", rtrn = False)

In [ ]:
b

In [ ]:
a[:128]

In [ ]:
min(2, 3)

In [ ]:
from modules.models import arima_hybrid

pr = arima_hybrid.get_arima_timegpt_predictions(data=a, prediction_horizon=1, frequency="daily", error_train_size = 64)

In [ ]:
pr

In [ ]:
import numpy as np
np.array([1, 2, 3]) - np.array([4, 5, 6])

In [79]:
pd.set_option('display.max_rows', 10)

In [15]:
from modules.data import data_loader

#a = data_loader.get_data(type = "commodity", ticker = "WTI", frequency = "weekly", start = "2019-01-01", end = "2024-01-01", rtrn = True)
#a = data_loader.get_data(type = "stock", ticker = "IBM", frequency = "daily", start = "01-01-2023", end = "01-01-2024")

In [16]:
b = data_loader.get_data(type = "fx", ticker = "USD/GBP", frequency = "weekly", start = "2015-01-01", end = "2020-01-01", rtrn = True)

In [17]:
b

,ds,y
1,2015-01-09,0.011656
2,2015-01-16,0.000000
3,2015-01-23,0.011067
4,2015-01-30,-0.004049
5,2015-02-06,-0.012496
...,...,...
256,2019-11-29,-0.007394
257,2019-12-06,-0.014963
258,2019-12-13,-0.015427
259,2019-12-20,0.025390


In [1]:
import pandas as pd
c = pd.read_csv("results/prediction/T=WTI_FR=weekly_TOD=commodity_FO=5_CLTS=32_SD=2019-01-01_ED=2024-01-01_FTL=200_FTF=5_FTG=0_TSCVR=6_BS=20_ME=4.csv", index_col = 0)
d = pd.read_csv("results/evaluation/T=WTI_FR=weekly_TOD=commodity_FO=5_CLTS=32_SD=2019-01-01_ED=2024-01-01_FTL=200_FTF=5_FTG=0_TSCVR=6_BS=20_ME=4.csv", index_col = 0)

In [3]:
from modules.data import metrics

metrics.mean_equal_sign(c)

,mes
arima,0.866667
autoregressor,0.466667
prophet,0.366667
lag_llama,0.466667
ft_lag_llama,0.466667
timeGPT,0.433333
ft_timeGPT,0.500000


In [15]:
"abc" in "ajskfhabcahsh"

True

In [21]:
"""import os
import pandas as pd

# Paths to the folders
folder1 = "results/evaluation"
folder2 = "results/prediction"

# List all CSV files in the second folder
files_in_folder2 = [f for f in os.listdir(folder2) if f.endswith('.csv')]

for file in files_in_folder2:
    
    # Full paths to the files
    file_path1 = os.path.join(folder1, file)
    file_path2 = os.path.join(folder2, file)
    
    # Read the CSV files into dataframes
    df1 = pd.read_csv(file_path1, index_col=0)
    df2 = pd.read_csv(file_path2, index_col=0)
    try:
        mes = metrics.mean_equal_sign(df2)
    except ZeroDivisionError:
        print(file)
        break
    
    df_merged = df1.join(mes)
    
    
    # Save the updated dataframe back to the first folder
    df_merged.to_csv("results/evaluation_with_mes/"+file)

print("All files have been updated successfully.")"""

All files have been updated successfully.


In [ ]:
"""import os
import pandas as pd

# Paths to the folders
folder1 = "results/evaluation"
folder2 = "results/prediction"

# List all CSV files in the second folder
files_in_folder2 = [f for f in os.listdir(folder2) if f.endswith('.csv')]

for file in files_in_folder2:
    if "TOD=cc" in file:
        continue
    # Full paths to the files
    file_path1 = os.path.join(folder1, file)
    file_path2 = os.path.join(folder2, file)
    
    # Read the CSV files into dataframes
    df1 = pd.read_csv(file_path1, index_col=0)
    df2 = pd.read_csv(file_path2, index_col=0)
    try:
        mes = metrics.mean_equal_sign(df2)
    except ZeroDivisionError:
        print(file)
        break
    
    df_merged = df1.join(mes)
    
    
    # Save the updated dataframe back to the first folder
    df_merged.to_csv("results/evaluation_with_mes/"+file)

print("All files have been updated successfully.")"""

In [8]:
a

,ds,y
1,2019-01-11,0.080426
2,2019-01-18,0.022450
3,2019-01-25,0.018490
4,2019-02-01,0.014183
5,2019-02-08,-0.001865
...,...,...
256,2023-12-01,-0.019865
257,2023-12-08,-0.063320
258,2023-12-15,-0.008485
259,2023-12-22,0.044501


In [7]:
from statsmodels.tsa.stattools import adfuller, acf, pacf
pacf(a["y"], nlags=len(a)//2 - 1)

array([ 1.00000000e+00, -6.18616706e-02, -4.19867743e-02,  6.98279699e-02,
        4.24912810e-02, -2.31593135e-03, -5.60290229e-02, -8.77903779e-02,
       -3.51976251e-02, -3.62018366e-02,  9.71060450e-03,  8.28603460e-03,
       -6.91259556e-03, -3.59815971e-02, -2.49966042e-02, -2.62490790e-02,
       -1.90792718e-02, -1.39096499e-03, -1.57950872e-02, -3.02366138e-02,
        1.32687974e-02, -1.29360540e-02, -6.00174951e-03, -2.43036566e-03,
       -8.95217317e-03, -6.03774081e-04, -3.41447796e-02,  5.85180692e-03,
        1.83835276e-02,  1.06312754e-02, -5.13947677e-03, -2.56214051e-02,
        6.16410492e-03,  7.30080507e-03,  5.26872738e-03, -1.78165810e-03,
        7.72739448e-03,  1.51210900e-02, -2.63554160e-02, -6.01056327e-03,
        9.56645248e-03,  1.49142905e-03,  1.39029956e-02, -6.26376911e-03,
        3.07570637e-03,  1.69115687e-02, -8.76883165e-03, -3.60830801e-02,
       -3.12340150e-02, -2.53231741e-02,  6.50392387e-03, -2.05947294e-03,
       -9.49195682e-03, -

In [3]:
from modules.data import time_series_analysis

time_series_analysis.analyze_time_series(a)

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\time_series_analysis.py:54: RuntimeWarning: invalid value encountered in double_scalars
  results['residual_volatility'] = volatility / noise_level


{'stationary': True,
 'adf_statistic': -17.05377860623053,
 'linear_trend': False,
 'linear_tau': -0.04015444015444015,
 'quadratic_trend': False,
 'quadratic_tau': 0.029937629937629932,
 'cyclical_patterns': False,
 'residual_mean': 0.0,
 'residual_volatility': nan}

In [60]:
x = data_loader.get_exogenous_data(start_date = "1987-01-01", end_date = "2024-01-01")

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values


In [69]:
from modules.models import timegpt

pr = timegpt.get_timegpt_forecast(data=a, prediction_length=1, frequency="monthly", x=x)

C:\Users\topco\AppData\Roaming\Python\Python311\site-packages\nixtlats\__init__.py:5: FutureWarning: This package is deprecated, please install nixtla instead.
  warnings.warn("This package is deprecated, please install nixtla instead.", category=FutureWarning)
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint.

In [70]:
pr

[3872.1965905683364]

In [27]:
# this adds partial evaluation rows to evaluation folder

"""import os
import pandas as pd

# Paths to the folders
folder1 = "results/evaluation"
folder2 = "results/partial/evaluation"

# List all CSV files in the second folder
files_in_folder2 = [f for f in os.listdir(folder2) if f.endswith('.csv')]

for file in files_in_folder2:
    # Full paths to the files
    file_path1 = os.path.join(folder1, file)
    file_path2 = os.path.join(folder2, file)
    
    # Read the CSV files into dataframes
    df1 = pd.read_csv(file_path1, index_col=0)
    df2 = pd.read_csv(file_path2, index_col=0)
    
    # Update df1 with rows from df2, replacing rows with the same index
    df1.update(df2)
    
    # Append the remaining rows from df2 that are not in df1
    df1 = df1.combine_first(df2)
    
    # Save the updated dataframe back to the first folder
    df1.to_csv(file_path1)

print("All files have been updated successfully.")"""


All files have been updated successfully.


In [31]:
"""import os
import pandas as pd

# Define paths
folder1 = "results/prediction"
folder2 = "results/partial/prediction"

# Get list of files in both folders
files_folder1 = set(os.listdir(folder1))
files_folder2 = set(os.listdir(folder2))

# Loop through the files in the second folder
for file_name in files_folder2:
    if file_name in files_folder1:
        # Load the dataframes
        df1 = pd.read_csv(os.path.join(folder1, file_name))
        df2 = pd.read_csv(os.path.join(folder2, file_name))
        
        merged_df = pd.merge(df1, df2, on="actual", how="inner", suffixes=('_df1', ''))

        # Drop the columns from df1 that have the same name as those in df2
        columns_to_drop = [col for col in merged_df.columns if col.endswith('_df1')]
        merged_df = merged_df.drop(columns=columns_to_drop)

        # Save the merged dataframe back to the first folder
        merged_df.to_csv(os.path.join(folder1, file_name), index=False)


"""

In [32]:
"""import os
import pandas as pd

# Define the path to the directory containing the CSV files
directory = "results/prediction"

# Iterate over each file in the directory
for filename in os.listdir(directory):
    # Check if the file is a CSV
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if "Unnamed: 0" column exists
        if "Unnamed: 0" in df.columns:
            # Set "Unnamed: 0" as the index and remove its name
            df.set_index("Unnamed: 0", inplace=True)
            df.index.name = None
            
            # Save the modified DataFrame back to the same file
            df.to_csv(file_path)
"""

In [ ]:
x = data_loader.get_exogenous_data(start_date="1920-01-01", end_date="2024-01-01")

In [ ]:
x

In [ ]:
import pandas as pd
import os
pd.read_csv(os.path.join("data", "credit_card", "uk_credit_card_data_monthly.csv"))


In [23]:
# This is for seeing if the evaluation and prediction contain the same files

import os

# Define the paths to the folders
evaluation_folder = "results/evaluation"
prediction_folder = "results/prediction"

# List the files in each folder
evaluation_files = set(os.listdir(evaluation_folder))
prediction_files = set(os.listdir(prediction_folder))

# Find files that are in evaluation but not in prediction
evaluation_not_in_prediction = evaluation_files - prediction_files

# Find files that are in prediction but not in evaluation
prediction_not_in_evaluation = prediction_files - evaluation_files

# Print the results
if evaluation_not_in_prediction:
    print("Files in 'evaluation' but not in 'prediction':")
    for file in evaluation_not_in_prediction:
        print(file)
else:
    print("No files are exclusively in 'evaluation'.")

if prediction_not_in_evaluation:
    print("\nFiles in 'prediction' but not in 'evaluation':")
    for file in prediction_not_in_evaluation:
        print(file)
else:
    print("No files are exclusively in 'prediction'.")


No files are exclusively in 'evaluation'.
No files are exclusively in 'prediction'.


In [ ]:


# this is for renaming files

#import os
'''def rename_files_in_folder(folder_path, old_string, new_string):
    try:
        # Get the list of all files in the specified folder
        files = os.listdir(folder_path)
        
        for filename in files:
            # Check if the old string is in the filename
            if old_string in filename:
                # Create the new filename by replacing the old string with the new string
                new_filename = filename.replace(old_string, new_string)
                
                # Construct the full file paths
                old_file_path = os.path.join(folder_path, filename)
                new_file_path = os.path.join(folder_path, new_filename)
                
                # Rename the file
                os.rename(old_file_path, new_file_path)
                
                print(f"Renamed: {filename} -> {new_filename}")
            else:
                print(f"No match found in: {filename}")
                
        print("Renaming process completed.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

folder_path = os.path.abspath(os.path.join(os.getcwd(), "results", "partial", "prediction")) #"/results/evaluation"
folder_path_2 = os.path.abspath(os.path.join(os.getcwd(), "results", "partial", "evaluation"))

s1 = "__"
s2 = "_"

rename_files_in_folder(folder_path, s1, s2)
rename_files_in_folder(folder_path_2, s1, s2)'''

In [26]:
"""# this is for adding something at the end

import os

# this is for renaming files

def rename_files_in_folder(folder_path):
    try:
        # Get the list of all files in the specified folder
        files = os.listdir(folder_path)
        
        for filename in files:
            # Check if the old string is in the filename
            
                old_file_path = os.path.join(folder_path, filename)
                new_file_path = old_file_path[:-4] + "_LR=0.0005.csv"
                
                # Rename the file
                os.rename(old_file_path, new_file_path)
                
                
                
        print("Renaming process completed.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

#folder_path = os.path.abspath(os.path.join(os.getcwd(), "results", "prediction")) #"/results/evaluation"
folder_path_2 = os.path.abspath(os.path.join(os.getcwd(), "results", "evaluation_with_mes"))



#rename_files_in_folder(folder_path)
rename_files_in_folder(folder_path_2)"""

Renaming process completed.


In [27]:
5e-4 == 0.0005

True

In [ ]:
# this combines ft_timeGPT evaluation with previous results

'''import os
import pandas as pd

# Define the paths to the folders
folder1 = "results/prediction"
folder2 = "results/partial/prediction"

# Get a list of all .csv files in both folders
files1 = [f for f in os.listdir(folder1) if f.endswith('.csv')]
files2 = [f for f in os.listdir(folder2) if f.endswith('.csv')]

# Ensure that only matching files are processed
matching_files = set(files1) & set(files2)

# Loop through each matching file
for filename in matching_files:
    # Construct the full file paths
    file1_path = os.path.join(folder1, filename)
    file2_path = os.path.join(folder2, filename)
    
    # Read the dataframes from both files
    df1 = pd.read_csv(file1_path, index_col=0)
    df2 = pd.read_csv(file2_path, index_col=0)
    
    # Concatenate the dataframes, preserving the index
    combined_df = pd.merge(df1, df2, on=['actual', 'timestamp'])
    
    # Save the combined dataframe back to the file in the first folder
    combined_df.to_csv(file1_path)'''

# Parameters

In [ ]:
# experiment parameters
#constraint: FOLDS x PREDICTION_LENGTH + TRAIN_SIZE !!must not!! be bigger than len(data)

PREDICTION_LENGTH = 1 # currentlz only works for PREDICTION_LENGTH > 1
TICKER = "S&P 500"
FREQUENCY = "minutely" # currently we only have dailz frequency
TYPE_OF_DATA = "stock" # currently we only have stock prices saved
MODELS = ["arima", "llama", "autoregressor", "fine-tuned Llama"] # currentlz works onlz for these two
FOLDS = 5 # for TSCV # reduced to two for testing purposes
CONTEXT_LENGTH = 245 # set to 245 for testing purposes
METRICS = ['r2', 'mse', 'mae', 'rmse', 'mda', "mape"]

# fine-tuning parameters
BATCH_SIZE = 10
MAX_EPOCHS = 5

# data parameters
START_DATE = "2024-07-22"
END_DATE = "2024-07-23"

# want to add
#TRAIN_PERIOD = # context lenghts. Should take a look into this
TRAIN_SIZE = CONTEXT_LENGTH



# Data

In [ ]:
"""
data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY)
# this is just in case there are more CSVs of the same type and frequency, the data should be the first in the list
#if len(data) > 1:
data = data[0]

simple_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["simple"])
#if len(simple_data) > 1:
simple_data = simple_data[0]

train_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["train"])
train_data = train_data[0]

test_data = data_reader.read_data(type = TYPE_OF_DATA, frequency = FREQUENCY, match = ["test"])
test_data = test_data[0]
"""

In [ ]:
DATA_CONFIG = {"ticker" : TICKER,
               "frequency" : FREQUENCY,
               "start" : START_DATE,
               "end" : END_DATE}



In [ ]:
data = data_loader.get_data(data_type=TYPE_OF_DATA, kwargs=DATA_CONFIG)

In [ ]:
data

In [ ]:
import 

# Additional parameters

In [ ]:
DATA_LENGTH = len(data)
FOLDS = int((DATA_LENGTH - TRAIN_SIZE) / PREDICTION_LENGTH) # this calculates max ammount of folds we can have given a set TRAIN_SIZE and PREDICTION_LENGTH
FOLDS = 5


In [ ]:
from modules.data import data_loader

In [ ]:
config = {"ticker" : "USD/GBP",
          "frequency" : "daily",
          "start" : "2024-07-01",
          "end" : "2024-07-30"}

In [ ]:
import os
print(os.getenv("ALPHAVANTAGE_API"))

In [ ]:
os.getenv("ALPHAVANTAGE_API")

In [ ]:
os.environ

In [ ]:
for key, value in os.environ.items():
    print(f'{key}: {value}')

In [ ]:
h = data_loader.get_data(data_type="exchange_rate", kwargs=config)

In [ ]:
h

# Fine tuning

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

In [ ]:
r, p = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY,
                           predictor=predictor)

# Saving results

In [ ]:
r

In [ ]:
p

In [ ]:
experiment_name = f"PREDICTION_LENGTH={PREDICTION_LENGTH}__TICKER={TICKER}__FREQUENCY={FREQUENCY}__TYPE_OF_DATA={TYPE_OF_DATA}__FOLDS={FOLDS}__CONTEXT_LENGTH/TRAIN_SIZE={CONTEXT_LENGTH}__FT_START_DATE={FT_START_DATE}__START_DATE={START_DATE}__END_DATE={END_DATE}__FT_LENGTH={FT_LENGTH}__DATA_LENGTH={DATA_LENGTH}" 

In [ ]:
experiment_name = f"P_L={PREDICTION_LENGTH}__T={TICKER}__FR={FREQUENCY}__T_O_D={TYPE_OF_DATA}__FO={FOLDS}__C_L_T_S={CONTEXT_LENGTH}__FT_S_D={FT_START_DATE}__S_D={START_DATE}__E_D={END_DATE}__FT_L={FT_LENGTH}__D_L={DATA_LENGTH}.csv"

In [ ]:
experiment_name

In [ ]:
result_saver.save_results(r, experiment_name, type="evaluation")
result_saver.save_results(p, experiment_name, type="prediction")

# Models

In [ ]:
# arima


In [ ]:
# lag llama

# Fine tuning an estimator

In [ ]:
PREDICTION_LENGTH = 5

In [ ]:
#preparing the training data for lag llama fine tuning
ft_train_data = lag_llama.prepare_data(data=ft_data, 
                                       prediction_length=0, 
                                       frequency=FREQUENCY)

In [ ]:
# creating the lag llama predictor object 
predictor = lag_llama_ft.get_predictor(prediction_length=PREDICTION_LENGTH, 
                                       context_length=CONTEXT_LENGTH, 
                                       batch_size=BATCH_SIZE, 
                                       max_epochs=MAX_EPOCHS)

In [ ]:
# fine-tuning the predictor object
predictor = predictor.train(ft_train_data, 
                            cache_data = True, 
                            shuffle_buffer_length = 1000)

# SKLearn TSCV

In [ ]:
r, p, a = get_tscv_results(data = data,
                           prediction_horizon=PREDICTION_LENGTH,
                           context_length=CONTEXT_LENGTH, 
                           folds=FOLDS, 
                           frequency=FREQUENCY, 
                           predictor=predictor)

In [ ]:
#creating the summary for each model
s = [get_summary(r[i]) for i in range(len(r))]

In [ ]:
means_df, medians_df, stds_df = extract_metrics(s, MODELS)

In [ ]:
means_df

# Visualisation

In [ ]:
graphs.standard_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.interactive_visualisation(MODELS, METRICS, s)

In [ ]:
graphs.prediction_visualisation(MODELS, p, a)

LABEL the graph
label the axes with timestamps
show the history of the actual
table of comparison, not just graph